In [97]:
from  bs4 import  BeautifulSoup 
import requests
import re
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)

In [98]:
def turn_to_url(snum):
    return 'https://www.truedorktimes.com/survivor/boxscores/s'+snum+ '.htm'

def process(string):
    return re.sub("\*", '', string)

#make sure they are in order or consider changing to a dict 
seasons = ['Borneo', 'The Australian Outback', 'Africa', 'Marquesas', 'Thailand', 'The Amazon', 
           'Pearl Islands', 'All-Stars', 'Vanuatu', 'Palau', 'Guatemala', 'Panama', 
           'Cook Islands', 'Fiji', 'China', 'Micronesia', 'Gabon', 'Tocantins', 'Samoa', 
           'Heroes vs. Villains', 'Nicaragua', 'Redemption Island', 'South Pacific',
           'One World', 'Philippines', 'Caramoan', 'Blood vs. Water', 'Cagayan', 'San Juan del Sur',
           'Worlds Apart', 'Cambodia', 'Kaôh Rōng', 'Millennials vs. Gen X',
           'Game Changers', 'Heroes vs. Healers vs. Hustlers', 'Ghost Island','David vs. Goliath']

In [99]:
def scrape_true_dork_scoring():
    data = []
    snum = 0
    macrolabels = ['Challenge Stats','Tribal Council Stats','Jury Stats', 'Survival Stats']
    labels = ['Contestant', 'ChW', 'ChA', 'ChW%', 'VFB', 'VAP', 'TotV', 
              'TCA', 'TC%', 'wTCR', 'JVF', 'TotJ', 'JV%', 'SurvSc', 'SurvAv','ID']
    def get_id(ssn,place):
        return str(ssn).zfill(2)+str(place).zfill(2)
    
    data.append(labels)
    for season in seasons:
        information = []
        snum+=1
        card = requests.get(turn_to_url(str(snum)))

        soup = BeautifulSoup(card.content, 'lxml')
        summary_table = soup.select('table#boxscore')[0].select('tr')
        
        i = len(summary_table[2].select('td'))
        #use for dynamic labeling(shouldnt be needed)
        #for y in range(i):
        #    labels.append(summary_table[1].select('th')[y].text)
        
        for x in range(2,len(summary_table)):
            info = []
            for y in range(i):
                if i == 15:
                    info.append(summary_table[x].select('td')[y].text)
                else:
                    if y != 4:
                        info.append(summary_table[x].select('td')[y].text)
            #print(snum,(x-1))
            dummy = str(snum)#get_id(snum,(x-1))
            info.append(dummy)
            data.append(info)
            
    rr = pd.DataFrame(data)
    rr.columns = rr.iloc[0]
    rr.drop([0],inplace=True)
    
    return rr

rz = scrape_true_dork_scoring()

In [100]:
ids = pd.read_csv('/home/analytics/Survivor/ids.csv',index_col=0)
ids.dtypes

name             object
season_number     int64
id                int64
dtype: object

In [101]:
def match_ids(rv):
    rv['NewID'] = None
    ids = pd.read_csv('/home/analytics/Survivor/ids.csv',index_col=0)
    group_ids = ids.groupby('season_number')
    group_data = rv.groupby('ID')
    replacement_dict = {'Sue':'Susan', 'BobDawg':'Bobby','Flica':'Jessica',
                    'Fairplay':'Jon','Boston Rob':'Rob','Purple Kelly':'Kelly S',
                    'Wendy Jo':'Wendy','Papa Bear':'Mark','Jeff Kent!':'Jeff',
                    'Laura M.':'Laura M','Laura B.':'Laura B','Big Tom':'Tom',
                    'Sarge':'Lea M','Bubba':'Travis'}

    corrected_doubles = {'Dan':1211,'Matt':2207,'Kat':2714,'Woo':2802,'So':3018,
                  'Taylor':3312,'Brad':3402,'Stephanie ':3616,'John':516}

    for x in group_data.groups.keys():
        #print(x)
        old_ids = group_ids.get_group(int(x))
        new_ids = group_data.get_group(x)
        #print(new_ids)
        #print(new_ids.index.values)
        for y in new_ids.index.values:
            posid = []
            #print(y)
            name = str(process(str(rv.iloc[(int(y)-1),0])))
            #print(name,'new')
            for z in old_ids.index.values:
                old_name = str(ids.iloc[int(z),0])
                #print(old_name,'old')
                if name in old_name:
                    posid.append(ids.iloc[int(z),2])
            if len(posid) == 1:
                #print('success',name,posid)
                rv.iloc[(int(y)-1),16] = str(posid[0]).zfill(4)
            elif len(posid) > 1:
                rv.iloc[(int(y)-1),16] = str(corrected_doubles[name]).zfill(4)
            else:
                #print('#####################error ', y, name, x, posid)
                if name in replacement_dict:
                    name2 = replacement_dict[name]
                else:
                    name2 = name.rstrip()
                    #print('################doubleerror ', y, name, x, posid)
                #print(name2,old_name)
                for z in old_ids.index.values:
                    old_name = str(ids.iloc[int(z),0])
                    if name2 in old_name:
                        posid.append(ids.iloc[int(z),2])
                        #print(posid)
                        if len(posid) == 1:
                            #print('success',name2,posid)
                            rv.iloc[(int(y)-1),16] = str(posid[0]).zfill(4)
    rt = rv
    rt.drop('ID',axis=1,inplace=True)
    rt.rename(index=str, columns={'NewID':'ID'},inplace=True)
    rt.set_index('ID',inplace=True)
    rt.sort_index(inplace=True)
    return rt

In [102]:
all_data = match_ids(rz).copy(deep=True)

In [103]:
def rename_contestants_multi_column(df,id2):
    def add_0(row):
        return str(row['id']).zfill(4)
    id2['id'] = id2.apply(add_0,axis=1)
    id2.set_index('id',inplace=True)
    id2.sort_index(inplace=True)
    
    #correct for skupin s2
    df.iloc[26,5:8] = ['0','15','2']
    #correct for jenna s8
    df.iloc[127,5:8] = ['0','0','0']
    #correct for janu s10
    df.iloc[155,5:8] = ['1','18','2']
    #correct for bruce s12
    df.iloc[192,5:8] = ['2','34','4']
    #correct for gary s14
    df.iloc[237,5:8] = ['0','0','0']
    #correct for james s16
    df.iloc[263,5:8] = ['3','46','5']
    #correct for kathy s16
    df.iloc[269,5:8] = ['0','19','2']
    #correct for penner s16
    df.iloc[271,5:8] = ['0','19','2']
    #correct for joe s18
    df.iloc[304,5:8] = ['1','26','4']
    #correct for russell swan s19
    df.iloc[324,5:8] = ['0','10','1']
    #correct for mike s19
    df.iloc[329,5:8] = ['0','10','1']
    #correct for kelly s21
    df.iloc[358,5:8] = ['0','58','6']
    #correct for naonka s21
    df.iloc[359,5:8] = ['3','58','6']
    #correct for colton s24
    df.iloc[419,5:8] = ['1','24','3']
    #correct for kourtney s24
    df.iloc[424,5:8] = ['0','0','0']
    #correct for dana s25
    df.iloc[438,5:8] = ['0','0','0']
    #correct for colton s27
    df.iloc[481,5:7] = ['0','10']
    #correct for lindsey s28
    df.iloc[495,5:7] = ['0','7']
    #correct for julie s29
    df.iloc[512,5:8] = ['2','9','1']
    #correct for joe s32
    df.iloc[561,5:8] = ['1','59','8']
    #correct for neal s32
    df.iloc[567,5:8] = ['0','6','1']
    #correct for caleb s32
    df.iloc[571,5:8] = ['0','0','0']
    #correct for bi s37
    df.iloc[669,5:8] = ['0','9','1']
    #correct for pat s37
    df.iloc[672,5:8] = ['0','0','0']


    
    for x in df.index.values:
        df.loc[x, 'Contestant'] = id2.loc[x,'name']
    display = df.copy(deep=True)    
    columns = [('','Contestant'),('Challenge Stats','ChW'),('Challenge Stats','ChA'),('Challenge Stats','ChW%'),
               ('Tribal Council Stats','VFB'),('Tribal Council Stats','VAP'),('Tribal Council Stats','TotV'),
               ('Tribal Council Stats','TCA'),('Tribal Council Stats','TC%'),('Tribal Council Stats','wTCR'),
               ('Jury Stats','JVF'),('Jury Stats','TotJ'),('Jury Stats','JV%'),
               ('Survival Stats','SurvSc'),('Survival Stats','SurvAv')]
    display.columns=pd.MultiIndex.from_tuples(columns)
               
    return df,display

final,display = rename_contestants_multi_column(all_data,ids)

final.to_csv('/home/analytics/Survivor/SurvivalScores.csv', index=True)
display

Challenge Stats               \
                      Contestant             ChW    ChA  ChW%   
ID                                                              
0101               Richard Hatch            1.87  15.10  0.12   
0102           Kelly Wiglesworth            5.87  15.10  0.39   
0103                 Rudy Boesch            1.62  15.10  0.11   
0104                  Susan Hawk            0.87  14.10  0.06   
0105                Sean Kenniff            1.87  13.10  0.14   
0106             Colleen Haskell            1.56  11.07  0.14   
0107            Gervase Peterson            2.89   9.07  0.32   
0108                 Jenna Lewis            1.23   7.07  0.17   
0109                   Greg Buis            2.56   5.07  0.50   
0110              Gretchen Cordy            1.23   3.07  0.40   
0111                   Joel Klug            0.89   2.07  0.43   
0112                   Dirk Been            0.62   1.51  0.41   
0113                 Ramona Gray            0.39   0.99  0.40   
0114             Stacey Stillman            0.29   0.68  0.42   
0115               B.B. Andersen            0.25   0.39  0.64   
0116           Sonja Christopher            0.00   0.13  0.00   
0201                 Tina Wesson            0.59  13.09  0.05   
0202             Colby Donaldson            8.09  13.09  0.62   
0203                 Keith Famie            2.59  13.09  0.20   
0204          Elisabeth Filarski            0.97  12.03  0.08   
0205              Rodger Bingham            0.57  10.03  0.06   
0206                Amber Brkich            0.59   9.09  0.07   
0207                  Nick Brown            1.97   7.03  0.28   
0208               Jerri Manthey            2.09   6.09  0.34   
0209              Alicia Calaway            0.97   4.53  0.21   
0210                 Jeff Varner            0.97   2.53  0.38   
0211              Michael Skupin            0.97   1.53  0.64   
0212            Kimmi Kappenberg            0.57   1.33  0.43   
0213              Mitchell Olson            0.39   0.99  0.40   
0214             Maralyn Hershey            0.39   0.66  0.59   
0215                 Kel Gleason            0.25   0.38  0.67   
0216                  Debb Eaton            0.00   0.13  0.00   
0301                  Ethan Zohn            2.87  13.29  0.22   
0302                 Kim Johnson            2.87  13.29  0.22   
0303          Lex van den Berghe            5.87  13.29  0.44   
0304                Tom Buchanan            1.67  12.29  0.14   
0305               Teresa Cooper            1.90  11.27  0.17   
0306                  Kim Powers            0.90   9.27  0.10   
0307              Frank Garrison            1.40   7.27  0.19   
0308             Brandon Quinton            1.40   6.27  0.22   
0309             Kelly Goldsmith            0.87   4.79  0.18   
0310              Clarence Black            0.87   2.79  0.31   
0311             Lindsey Richter            0.90   1.77  0.51   
0312               Silas Gaither            0.70   1.37  0.51   
0313               Linda Spencer            0.27   1.04  0.26   
0314             Carl Bilancione            0.39   0.73  0.54   
0315              Jessie Camacho            0.00   0.39  0.00   
0316                 Diane Ogden            0.00   0.13  0.00   
0401              Vecepia Towery            2.45  15.35  0.16   
0402                Neleh Dennis            2.34  15.33  0.15   
0404             Paschal English            5.51  15.33  0.36   
0404       Kathy Vavrick-O'Brien            2.84  14.33  0.20   
0405                 Sean Rector            1.75  13.35  0.13   
0406              Robert DeCanio            1.62  11.33  0.14   
0407               Tammy Leitner            3.01   9.33  0.32   
0408               Zoe Zanidakis            0.96   7.33  0.13   
0409                John Carroll            0.56   5.83  0.10   
0410                 Rob Mariano            0.25   3.85  0.06   
0411                  Gina Crews            0.75   2.85  0.26   
041

0
Contestant    Pat Cusack
ChW                 0.00
ChA                 0.10
ChW%                0.00
VFB                    0
VAP                    0
TotV                   0
TCA                    0
TC%                -1.00
wTCR                0.00
JVF                    -
TotJ                   -
JV%                    -
SurvSc             -1.00
SurvAv              0.00
Name: 3720, dtype: object

In [105]:
final

,Contestant,ChW,ChA,ChW%,VFB,VAP,TotV,TCA,TC%,wTCR,JVF,TotJ,JV%,SurvSc,SurvAv
ID,,,,,,,,,,,,,,,
0101,Richard Hatch,1.87,15.10,0.12,10,6,73,11,0.90,2.55,4,7,0.57,1.60,7.84
0102,Kelly Wiglesworth,5.87,15.10,0.39,6,0,73,11,0.55,3.82,3,7,0.43,1.36,12.26
0103,Rudy Boesch,1.62,15.10,0.11,10,8,73,11,0.90,2.12,-,-,-,1.01,3.74
0104,Susan Hawk,0.87,14.10,0.06,9,5,70,10,0.89,2.80,-,-,-,0.95,3.67
0105,Sean Kenniff,1.87,13.10,0.14,6,9,66,9,0.65,1.44,-,-,-,0.79,3.30
0106,Colleen Haskell,1.56,11.07,0.14,3,7,61,8,0.36,0.95,-,-,-,0.50,2.51
0107,Gervase Peterson,2.89,9.07,0.32,1,6,55,7,0.13,0.40,-,-,-,0.45,3.29
0108,Jenna Lewis,1.23,7.07,0.17,4,11,48,6,0.63,1.24,-,-,-,0.80,2.47
0109,Greg Buis,2.56,5.07,0.50,1,6,40,5,0.17,0.56,-,-,-,0.67,3.12
